In [1]:
import os
import shutil
import time

import torch

from envs.IBGym_mod_envs import IBGymQ
from offline.convert_datset import create_lstm_dataset, create_look_ahead_lstm_dataset
from offline.dataset_creater import GymEnvSampler
from state_quantization.transforms import NormalizeTransform
from offline.convert_datset import merge_rllib_out, save_numpy
import multiprocessing as mp

# Env Creator

In [2]:


device = 'cuda'


model_path = 'dynamic_model/model'

model = torch.load(model_path).to(device)
model.eval()
model.look_ahead = 0

normalize_dataset = NormalizeTransform.load('dynamic_model/NormalizeInputConfigs.pkl')
normalize_dataset.to(device)

def env_creator(steps_per_episode=1000):
    return IBGymQ(q_model=model,device='cuda', setpoint=70, reward_type='classic', action_type='discrete', observation_type='include_past',
             reset_after_timesteps=steps_per_episode, n_past_timesteps=model.seq_len, normalize_transformer=normalize_dataset)

# Generate Episodes data

In [3]:
episodes = 100
steps_per_episode = 1000

In [4]:
writer_path = os.path.join("tmp", "ibq-out")
if os.path.exists(writer_path) and os.path.isdir(writer_path):
    shutil.rmtree(writer_path)
start = time.time()
env = env_creator(steps_per_episode)
env_sampler = GymEnvSampler(env, writer_path, episodes=episodes)
env_sampler.create_dataset()
end = time.time()
print(end-start)

/home/hamza/PycharmProjects/StateCompression/venv/lib/python3.8/site-packages/gym/spaces/box.py:73: UserWarning: WARN: Box bound precision lowered by casting to float32
  logger.warn(
2022-09-29 21:47:40,892	WARNING deprecation.py:46 -- DeprecationWarning: `SampleBatchBuilder` has been deprecated. Use `a child class of `SampleCollector`` instead. This will raise an error in the future!


Episode: 0
Episode: 1
Episode: 2
Episode: 3
Episode: 4
Episode: 5
Episode: 6
Episode: 7
Episode: 8
Episode: 9
Episode: 10
Episode: 11
Episode: 12
Episode: 13
Episode: 14
Episode: 15
Episode: 16
Episode: 17
Episode: 18
Episode: 19
Episode: 20
Episode: 21
Episode: 22
Episode: 23
Episode: 24
Episode: 25
Episode: 26
Episode: 27
Episode: 28
Episode: 29
Episode: 30
Episode: 31
Episode: 32
Episode: 33
Episode: 34
Episode: 35
Episode: 36
Episode: 37
Episode: 38
Episode: 39
Episode: 40
Episode: 41
Episode: 42
Episode: 43
Episode: 44
Episode: 45
Episode: 46
Episode: 47
Episode: 48
Episode: 49
Episode: 50
Episode: 51
Episode: 52
Episode: 53
Episode: 54
Episode: 55
Episode: 56
Episode: 57
Episode: 58
Episode: 59
Episode: 60
Episode: 61
Episode: 62
Episode: 63
Episode: 64
Episode: 65
Episode: 66
Episode: 67
Episode: 68
Episode: 69
Episode: 70
Episode: 71
Episode: 72
Episode: 73
Episode: 74
Episode: 75
Episode: 76
Episode: 77
Episode: 78
Episode: 79
Episode: 80
Episode: 81
Episode: 82
Episode: 83
Ep

In [5]:


data_path = os.path.join("tmp", "ibq-out/*.json")
merged_data = merge_rllib_out(data_path)
save_path = os.path.join("tmp", "ibq-out/ibq_samples.npy")
save_numpy(save_path,merged_data)